In [1]:
IMAGE_PATH = "/Users/antoinegerardin/Documents/data/rt-cetsa/Data for Nick/20210318 LDHA compound plates/20210318 LDHA compound plate 1 6K cells/1.tif"

from bfio import BioReader
from matplotlib import pyplot as plt

with BioReader(IMAGE_PATH ,backend='bioformats') as br:
    data = br[:]
    plt.imshow(data, interpolation='nearest')
    plt.show()


ModuleNotFoundError: No module named 'matplotlib'